In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
pip install pypdf -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
def load_document(file):
    from langchain.document_loaders import PyPDFLoader
    
    